<div style="text-align:center"><span style="color:blue; font-family:Times New Roman; font-size:3em;"> Convolutional neural network modeling </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Convolutional neural networks have unique architecture and are reportedly successful at image recognition. Because my laptop always got frozen while training a CNN, I decide to use Google Cloud Datalab which is an interactive tool for Jupyter notebooks. The Datalab is connected to a Google Compute Engine Virtual Machine (VM) with 16 CPU for massive data training. The image input data and output variables for the training and test datasets are preprocessed, saved in Google Cloud Storage, and downloaded to Datalab. x_data and y_train are the input and output variables of the training dataset. x_test and x_test_ID are the input data and image IDs for the test dataset. 
 </span></div>

In [1]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in c:\python27\scripts\anaconda2\lib\site-packages


In [2]:
!pip install keras

    100% |################################| 204kB 2.9MB/s 
    100% |################################| 3.1MB 396kB/s 
  Running setup.py bdist_wheel for keras ... - done
  Stored in directory: /root/.cache/pip/wheels/48/82/42/f06a8c03a8f95ada523a81ba723e89f059693e6ad868d09727
  Running setup.py bdist_wheel for theano ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/d5/5b/93/433299b86e3e9b25f0f600e4e4ebf18e38eb7534ea518eba13
Successfully built keras theano


In [3]:
!pip install -U h5py==2.6

    100% |################################| 4.2MB 310kB/s 
    100% |################################| 16.5MB 65kB/s 
Requirement already up-to-date: six in /usr/local/lib/python2.7/dist-packages (from h5py==2.6)
  Found existing installation: numpy 1.11.2
    Uninstalling numpy-1.11.2:
      Successfully uninstalled numpy-1.11.2


In [4]:
!sed -i.bak '/run_tests/d' /usr/local/lib/python2.7/dist-packages/h5py/__init__.py

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from pandas import DataFrame, Series
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.externals import joblib
from StringIO import StringIO
from io import BytesIO 
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers.convolutional import Convolution2D 
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import RMSprop
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras import backend as K
K.set_image_dim_ordering('th')
import h5py

Using TensorFlow backend.


In [2]:
%storage read --object gs://image1234/x_data_32.pkl --variable x_data

In [3]:
x_data=joblib.load(BytesIO(x_data), mmap_mode='c')
x_data.shape

(1409, 3, 32, 32)

In [4]:
%storage read --object gs://image1234/y_train_colorscaled.pkl --variable y_train_colorscaled

In [5]:
y_train=joblib.load(BytesIO(y_train_colorscaled), mmap_mode='c')
y_train.shape

(1409,)

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
I use k-Fold Cross Validation to estimate the performance of a model on unseen data. The training dataset is split into 10 subsets. Each time, a model will train on all subsets except one which is held out as validation set. Training the model 10 times on 90% of the data and testing on 10% can provide a robust estimate of model performance.


 </span></div>

In [6]:
kfolds=KFold(n_splits=10, shuffle=True,random_state=0)
trainindex=[]
validindex=[]
for train_index, valid_index in kfolds.split(x_data, y_train):
    trainindex.append(train_index)
    validindex.append(valid_index)
trainindex=np.array(trainindex)
validindex=np.array(validindex)

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
The network structure consists of at least 3 convolutional layers. Additional 3 convolutional layers can be added. Each convolutional layer has a filter size of 3 x 3 and a rectifier activation function. 
Every one or two convolutional layers will be followed by a max pool layer with a window size of 2×2.
Afterwards, the output of the third pooling layer is flattened to 1D as the Flatten layer, and passed through two fully connected Dense layers. Better performance is achieved using the tanh activation function in the first dense layer. 
The last dense layer has the same number of outputs as the number of the class types and a softmax activation is used for purposes of probabilistic classification.
<br\>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Two Dropout layers are applied after the third pooling layer, and after the first Dense layer. Dropout is able to randomly select nodes to be dropped-out with a given probability at each weight update cycle, which can achieve better generalization error and is less likely to overfit the training data. 
<br\>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
After the model is defined, I compile the model by specifying  'sparse_categorical_crossentropy' as the chosen loss function to evaluate a set of weights. I use AdaMax as an optimizer to search through different weights for the network because it performs better than other optimization methods.
<br\>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
Due to a limited amount data for deep learning training, I apply image augmentations to artificially increase the size of the training set with new transformed images. A number of random transformations are applied to the initial data by zooming, rotating, shifting or shearing images to prevent overfitting and improving the model generalization error. 
<br\>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
In the end, the model is fitted by specifying training data, validation data, the number of training epochs and the batch size. 
The model weights that give the best result (lowest loss in the validation set) is saved by ModelCheckpoint in the file 'weights32_testnumber.best.hdf5'. The best model weights can be reloaded for model evaluation and prediction.        

 </span></div>

In [7]:
def cnn_32(x_data, y_train,trainindex, validindex, foldnumber, testnumber, c1,c2,c3,c4,c5,c6,c7,nb_epoch ):
    X_train, X_valid = x_data[trainindex[foldnumber]], x_data[validindex[foldnumber]]
    y_train, y_valid = y_train[trainindex[foldnumber]], y_train[validindex[foldnumber]]
          
    model=Sequential()
    model.add(Convolution2D(nb_filter=c1, nb_row=3, nb_col=3, border_mode='same',\
                            input_shape=(3,32,32), activation='relu'))
    if c2 != 0:
        model.add(Convolution2D(nb_filter=c2, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    
    ############### 
    model.add(Convolution2D(nb_filter=c3, nb_row=3, nb_col=3, border_mode='same', \
                            activation='relu'))
    if c4 != 0:
        model.add(Convolution2D(nb_filter=c4, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
   
    ###############          
    model.add(Convolution2D(nb_filter=c5, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    if c6 != 0:
        model.add(Convolution2D(nb_filter=c6, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    
    ###############
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(c7, init='he_normal', activation = 'tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(3, init='he_normal', activation = 'softmax'))
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adamax',#optimizer='adadelta', 'adamax','rmsprop','adam',sgd,
                  metrics=['accuracy'])
    #print(model.summary())
    datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,
                                 zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')   
    filepath="weights32_{}.best.hdf5".format(testnumber)
    saveBestModel = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')  
     
    model.fit_generator(datagen.flow(X_train, y_train, batch_size=500, shuffle=True), \
                        nb_epoch=nb_epoch, samples_per_epoch=len(X_train), verbose=0, \
                        validation_data=(X_valid, y_valid),
                        callbacks=[saveBestModel])
    model.load_weights(filepath)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adamax',
                  metrics=['accuracy'])
    scores_valid = model.evaluate(X_valid, y_valid,verbose=0)
    scores_train = model.evaluate(X_train, y_train,verbose=0)
    print("%s in the train set: %.6f" % (model.metrics_names[0], scores_train[0]))
    print("%s in the train set: %.2f%%" % (model.metrics_names[1], scores_train[1]*100)) 
    print("%s in the validation set: %.6f" % (model.metrics_names[0], scores_valid[0]))
    print("%s in the validation set: %.2f%%" % (model.metrics_names[1], scores_valid[1]*100))    
    return scores_train[0], scores_valid[0]

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
10-fold train/validation combinations are iterated and trained in a CNN model. The model performance is evaluated by averaging the loss values in the 10-fold validation sets. For an example shown below, this CNN model consists of 4 convolutional layers with the filter numbers of 16, 16, 16 and 32. The output neurons of the first dense layer is 32. The average validation loss for this model is 0.77649 +/- 0.03402, which is much better than a logistic regression model presented earlier.


 </span></div>

In [9]:
train_scores=[]
valid_scores=[]
for k in range(0,10):
    print 'This is %d-th fold : ' %k
    scores_train, scores_valid= cnn_32(x_data, y_train,trainindex, validindex, k, k,  16,16,16,0,32,0,32,500)
    train_scores.append(scores_train)
     valid_scores.append(scores_valid)

This is 0-th fold : 
loss in the train set: 0.713650
acc in the train set: 68.45%
loss in the validation set: 0.795650
acc in the validation set: 60.28%
This is 1-th fold : 
loss in the train set: 0.733433
acc in the train set: 67.35%
loss in the validation set: 0.811755
acc in the validation set: 65.96%
This is 2-th fold : 
loss in the train set: 0.772045
acc in the train set: 65.46%
loss in the validation set: 0.744944
acc in the validation set: 69.50%
This is 3-th fold : 
loss in the train set: 0.743023
acc in the train set: 67.74%
loss in the validation set: 0.802167
acc in the validation set: 60.99%
This is 4-th fold : 
loss in the train set: 0.697524
acc in the train set: 68.85%
loss in the validation set: 0.791737
acc in the validation set: 63.83%
This is 5-th fold : 
loss in the train set: 0.710163
acc in the train set: 68.53%
loss in the validation set: 0.726430
acc in the validation set: 69.50%
This is 6-th fold : 
loss in the train set: 0.713938
acc in the train set: 68.06%


In [14]:
print 'The average loss in the train set is %.5f +/- %.5f' %(np.mean(train_scores),np.std(train_scores))
print 'The average loss in the validation set is %.5f +/- %.5f' %(np.mean(valid_scores),np.std(valid_scores))

The average loss in the train set is 0.73105 +/- 0.02592
The average loss in the validation set is 0.77649 +/- 0.03402


<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
After being through countless trials with different combinations of convolutional layer numbers, filter numbers in convolutional layers, output numbers in the first dense layer, and the epoch numbers, a best model is selected based on its average loss score. Finally, the test dataset is feed into the best model which generates 10 predictions via the iterations over 10-fold cross validations. The final prediction for the test set is obtained by averaging all 10 predictions in order to reduce overfitting. 

 </span></div>

In [15]:
%storage read --object gs://image1234/x_test_32.pkl --variable x_test

In [16]:
%storage read --object gs://image1234/x_test_ID.pkl --variable x_test_ID 

In [17]:
x_test=joblib.load(BytesIO(x_test), mmap_mode='c') 
x_test.shape

(512, 3, 32, 32)

In [18]:
x_test_ID =joblib.load(BytesIO(x_test_ID), mmap_mode='c')

In [20]:
def cnn_predict(x_test, x_data, y_train,trainindex, validindex, foldnumber, testnumber, c1,c2,c3,c4,c5,c6,c7):
    X_train, X_valid = x_data[trainindex[foldnumber]], x_data[validindex[foldnumber]]
    y_train, y_valid = y_train[trainindex[foldnumber]], y_train[validindex[foldnumber]]
          
    model=Sequential()
    model.add(Convolution2D(nb_filter=c1, nb_row=3, nb_col=3, border_mode='same',\
                            input_shape=(3,32,32), activation='relu'))
    if c2 != 0:
        model.add(Convolution2D(nb_filter=c2, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    
    ############### 
    model.add(Convolution2D(nb_filter=c3, nb_row=3, nb_col=3, border_mode='same', \
                            activation='relu'))
    if c4 != 0:
        model.add(Convolution2D(nb_filter=c4, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
   
    ###############          
    model.add(Convolution2D(nb_filter=c5, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    if c6 != 0:
        model.add(Convolution2D(nb_filter=c6, nb_row=3, nb_col=3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    
    ###############
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(c7, init='he_normal', activation = 'tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(3, init='he_normal', activation = 'softmax'))
  
    ###############
    filepath="weights32_{}.best.hdf5".format(testnumber)
    model.load_weights(filepath)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adamax',
                  metrics=['accuracy'])
    scores = model.evaluate(X_valid, y_valid,verbose=0)
    y_pred = model.predict_proba(x_test)
   
    print("%s in the validation set: %.6f" % (model.metrics_names[0], scores[0]))
    print("%s in the validation set: %.2f%%" % (model.metrics_names[1], scores[1]*100))    
    return y_pred
    
    
    

In [24]:
y_pred_all=[]
for k in range(0,10):
    y_pred=cnn_predict(x_test, x_data, y_train,trainindex, validindex, k, k,  16,16,16,0,32,0,32)
    y_pred_all.append(y_pred)

480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.795650
acc in the validation set: 60.28%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.811755
acc in the validation set: 65.96%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.744944
acc in the validation set: 69.50%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.802167
acc in the validation set: 60.99%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.791737
acc in the validation set: 63.83%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.726430
acc in the validation set: 69.50%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.723872
acc in the validation set: 66.67%
480/512 [===========================>..] - ETA: 0sloss in the validation set: 0.796617
acc in the validation set: 64.54%
480/512 [=======================

In [40]:
y_pred_all=np.array(y_pred_all)
y_pred_average=np.average(y_pred_all, axis=0)
df = pd.DataFrame(y_pred_average, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = x_test_ID
df = df.reindex(columns=['image_name','Type_1','Type_2','Type_3'])
df.to_csv('submission_a1.csv', index=False)

<div style="text-align:center"><span style="color:blue; font-family:Times New Roman; font-size:3em;"> Conclusion  </span></div>

<div style="text-align:justify"><span style="color:black; font-family:Times New Roman; font-size:1.5em;line-height:1.4em;"> 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
In this project, I developed two learning algorithms to classify cervix types.  A logistic regression model combined with different image feature extraction techniques is simple to implement and low computation cost. However, this classifier only can achieve a Log Loss  value of 0.86822. Convolutional neural networks are very expensive to train but shows superior performance. I have successfully implemented a multi-layer CNN in Keras with the combination of Dropout, image augmentations and k-fold cross validation in order to reduce overfitting and log-loss value . The best result of a log loss from CNN is 0.752 in the validation set and 0.758 in the test set, leading to the ranking of top 12% in the Kaggle competition ‘Cervical Cancer Screening’.
 </span></div>